### Scraping website using pandas

### Import Pandas Library

In [316]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib import colors

### Use pd.read_html() to get all the tables on the website 

In [317]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)

### We get a list of dataframe, if we have multiple tables on the website. Choose the required table from the list.

In [318]:
list_toronto = data[0]
list_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Ignoring rows with Not assigned values for column Borough .

In [319]:
list_toronto = list_toronto[list_toronto['Borough'] != 'Not assigned']
list_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Check the shape of the dataframe

In [320]:
list_toronto.shape

(103, 3)

### Install Geocoder

In [40]:
!pip install geocoder

     |████████████████████████████████| 102kB 1.7MB/s ta 0:00:011


### define method to get Coordinates

In [42]:
import geocoder


def get_location(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
        return lat_lng_coords
    


### Getting Coordinates using Geocoder

In [ ]:
post_code = list(list_toronto['Postal Code'].unique())
latlog = {}
for i in post_code:
    coords = get_location(i)
    latlog[i]=coords

### Above took extremely long time. So, alternative csv was used.

In [321]:
location_data = pd.read_csv("/Users/SujayaJU/Downloads/Geospatial_Coordinates.csv") ## if, running the code please give the path where you have stored the file
location_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Using pd.merge() to join two dataframes based on Postal Code. We will only concentrate on Torontro City

In [326]:
list_toronto_latlog = pd.merge(list_toronto, location_data, on='Postal Code')
list_toronto_latlog = list_toronto_latlog[list_toronto_latlog["Borough"].apply(lambda x: "Toronto" in x)]
list_toronto_latlog.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


### Get Co-Ordinates of Toronto

In [327]:
# Toronto coordinates

latitude = 43.651070
longitudnal = -79.347015

### Using Folium to mark the neighbourhood on the Map

In [324]:
!pip install folium
import folium

In [325]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitudnal], zoom_start=11)

# add markers to map
for lat, lng, label in zip(list_toronto_latlog['Latitude'], list_toronto_latlog['Longitude'], list_toronto_latlog['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto

### Connected to Foursquare. Credentials deleted for privacy

In [331]:
list_toronto_latlog.loc[2, 'Borough']
neighborhood_latitude = list_toronto_latlog.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = list_toronto_latlog.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = list_toronto_latlog.loc[2, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Downtown Toronto are 43.6542599, -79.3606359.


In [332]:
LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BL3RRBJZRBPUCP40MLO1JSIDZ1SNRYTHOYMXH5BJUDZPURQZ&client_secret=AHQRX1BXRX3EJPMWRPUQRHQEXAFOJOPQYPKANNPGEWEZ5OO5&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [333]:
results = requests.get(url).json()

In [334]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Exploring venues in Toronto, and their categories

In [335]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [336]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [337]:
toronto_venues = getNearbyVenues(names=list_toronto_latlog['Borough'],
                                   latitudes=list_toronto_latlog['Latitude'],
                                   longitudes=list_toronto_latlog['Longitude']
                                  )

Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
West Toronto
East Toronto
Downtown Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Central Toronto
West Toronto
Central Toronto
Downtown Toronto
West Toronto
Central Toronto
Downtown Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
East Toronto


In [338]:
toronto_venues.head()

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Downtown Toronto,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,Downtown Toronto,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Downtown Toronto,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Downtown Toronto,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [339]:
toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Central Toronto,112,112,112,112,112,112
Downtown Toronto,1242,1242,1242,1242,1242,1242
East Toronto,124,124,124,124,124,124
West Toronto,160,160,160,160,160,160


In [340]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = toronto_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Downtown Toronto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [341]:
toronto_grouped = toronto_onehot.groupby('Borough').mean().reset_index()
toronto_grouped

,Borough,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.008929,0.00000,...,0.008929,0.008929,0.000000,0.008929,0.000000,0.008929,0.000000,0.000000,0.000000,0.008929
1,Downtown Toronto,0.000805,0.000805,0.000805,0.000805,0.00161,0.002415,0.000805,0.013688,0.00161,...,0.000805,0.000805,0.002415,0.011272,0.000805,0.003221,0.006441,0.000805,0.000805,0.005636
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.024194,0.00000,...,0.008065,0.016129,0.000000,0.000000,0.000000,0.000000,0.008065,0.000000,0.000000,0.024194
3,West Toronto,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00625,...,0.000000,0.000000,0.000000,0.012500,0.000000,0.012500,0.006250,0.000000,0.000000,0.012500


In [342]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [343]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
2,East Toronto,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub
3,West Toronto,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant


### Using K-means Clustering on Boroughs of Toronto city, To find the most common venues in different part of Toronto city

In [344]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 3, 2, 1], dtype=int32)

In [345]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = list_toronto_latlog
toronto_merged.rename(columns={'Borough': 'Borough'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

In [346]:
toronto_merged.dropna(inplace=True)
toronto_merged["Cluster Labels"]=toronto_merged["Cluster Labels"].astype(int)
# create map
map_clusters = folium.Map(location=[latitude, longitudnal], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 0

In [347]:
y = toronto_merged.loc[toronto_merged["Cluster Labels"]==0,:]
print(y.shape)
y

(9, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
62,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
67,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
68,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
73,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
79,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,0,Coffee Shop,Sandwich Place,Pizza Place,Café,Park,Restaurant,Clothing Store,Sushi Restaurant,Italian Restaurant,Pub


### Cluster 1

In [348]:
y = toronto_merged.loc[toronto_merged["Cluster Labels"]==1,:]
print(y.shape)
y

(6, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant
43,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant
69,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant
75,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant
81,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,1,Café,Bar,Coffee Shop,Italian Restaurant,Bakery,Restaurant,Grocery Store,Breakfast Spot,Diner,Mexican Restaurant


### Cluster 2

In [349]:
y = toronto_merged.loc[toronto_merged["Cluster Labels"]==2,:]
print(y.shape)
y

(5, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub
47,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub
54,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,2,Greek Restaurant,Coffee Shop,Café,Italian Restaurant,Brewery,Restaurant,Ice Cream Shop,Yoga Studio,Park,Pub


### Cluster 3

In [350]:
y = toronto_merged.loc[toronto_merged["Cluster Labels"]==3,:]
print(y.shape)
y

(19, 16)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,3,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Park,Bakery,Gym,Clothing Store


### From the results we can see that the most common places,
1. We can have Japanese food at Central and Downtown Toronto.
2. We can have Greek food at East Toronto.
3. We can have Mexican food at West Toronto.
4. Gym's is avialable at Downtown Toronto and Yoga Studios at East Toronto